In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
!git clone https://github.com/Abisheka06/Solar_Panel_Dataset.git
%cd Solar_Panel_Dataset

Cloning into 'Solar_Panel_Dataset'...
remote: Enumerating objects: 809, done.
remote: Total 809 (delta 0), reused 0 (delta 0), pack-reused 809 (from 1)
Receiving objects: 100% (809/809), 269.00 MiB | 30.74 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (891/891), done.
/content/Solar_Panel_Dataset


In [3]:
import os
import shutil

source_dir = '/content/Solar_Panel_Dataset'
clean_dir = '/content/Clean_Solar_Panel_Dataset'

os.makedirs(clean_dir, exist_ok=True)

# Copy only folders that are actual class names (not starting with '.')
for item in os.listdir(source_dir):
    s = os.path.join(source_dir, item)
    d = os.path.join(clean_dir, item)
    if os.path.isdir(s) and not item.startswith('.'):
        shutil.copytree(s, d, dirs_exist_ok=True)


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

# Set paths
data_dir = "/content/Clean_Solar_Panel_Dataset"

# Image parameters
img_size = (224, 224)
batch_size = 32

# Data generators with split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
print(train_gen.class_indices)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Number of classes
num_classes = len(train_gen.class_indices)
print(num_classes)

# Build model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_gen, validation_data=val_gen, epochs=30)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"\nValidation Accuracy: {acc*100:.2f}%")

# Save the model
model.save("solar_panel_classifier.keras")
print("\nModel Saved")


Found 711 images belonging to 6 classes.
{'Bird-drop': 0, 'Clean': 1, 'Dusty': 2, 'Electrical-damage': 3, 'Physical-Damage': 4, 'Snow-Covered': 5}
Found 174 images belonging to 6 classes.
6
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.2354 - loss: 2.1467 - val_accuracy: 0.3506 - val_loss: 1.5918
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 976ms/step - accuracy: 0.2656 - loss: 1.9939 - val_accuracy: 0.3908 - val_loss: 1.4522
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.3571 - loss: 1.7150 - val_accuracy: 0.4655 - val_loss: 1.3337
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 929ms/step - accuracy: 0.4283 - loss: 1.5420 - val_accuracy: 0.5402 - val_loss: 1.2411
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 973ms/step - accuracy: 0.3949 - loss: 1.5693 - val_accuracy: 0.5517 - val_loss: 1.2102
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 971ms/step - accuracy: 0.4492 - loss: 1.4406 - val_accuracy: 0.5632 - val_loss: 1.1432
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 980ms/step - accuracy: 0.4340 - loss: 1.3955 - val_accuracy: 0.6264 - val_loss: 1.0981
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 22s 957ms/step - accuracy: 0.4796 - loss: 1.4210 - val_accuracy: 

In [5]:
from google.colab import files
files.download("solar_panel_classifier.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>